In [ ]:
# imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.gridspec as gridspec

%load_ext autoreload
%autoreload 1
import json

import h5py as h5
#import seaborn as sns
#plt.style.use('https://raw.githubusercontent.com/danionella/plotstyles/main/JJ.mplstyle')
#sns.set(font_scale=2)
#sns.set_style("white")
#sns.set_style("ticks")
from scipy.stats import pearsonr
from utils import load_from_h5, colorbar
%matplotlib widget

In [ ]:
def pfield2obs(pfield, startx, stopx, nx, starty, stopy, ny, rho):
    result = {}
    # turn into observables
    # step sizes
    dx = 1e-2 * (stopx - startx) / (nx - 1)  # start/stop in cm
    dy = 1e-2 * (stopy - starty) / (ny - 1)
    result["pressure"] = pfield  # dim: nsamples, ny, nx, nspeakers
    result["acceleration_y"] = - np.gradient(pfield, axis=1) / dy / rho  # Euler Eq.
    result["acceleration_x"] = - np.gradient(pfield, axis=2) / dx / rho  # Euler Eq.
    return result

In [ ]:
def accelfield2obs(accelfield):
    # transform from names on sensor to own name convention for axes
    # channel order: X,Y,Z as per label on of PCB acceleration sensor
    result = {}
    result["acceleration_x"] = -accelfield[:,:,:,2] # Z is L/R sp0/sp1 axis, sign on sensor points left towards sp0 
    result["acceleration_y"] = accelfield[:,:,:,1] # Y is along sp2/sp3 axis, sign on sensor points down towards sp3
    result["acceleration_z"] = -accelfield[:,:,:,0] # X is top/down axis, sign on sensor points to bottom
    return result

In [ ]:
def linebreaks(my_str, group=10, char='\n'):
    return char.join(my_str[i:i+group] for i in range(0, len(my_str), group))

# Filenames

In [ ]:
Xserver = r'./'
fn_pfields = glob.glob(os.path.join(Xserver,'field_constancy/*pfield*.h5'))
fn_accelfields = glob.glob(os.path.join(Xserver,'field_constancy/*accelfield*.h5'))
fn_stimset = r'./2023-03-23_18-57-01_field/2023-03-23_18-57-01stimset_field.h5'
out = r"./field_constancy/plots"

In [ ]:
ss = load_from_h5(fn_stimset)

In [ ]:
pfields = {}
for fn_pfield in fn_pfields:
    tmp = load_from_h5(fn_pfield)
    date = list(tmp['constancy'].keys())[0] #single entry
    pfields[date] = tmp['constancy'][date]

In [ ]:
accelfields = {}
for fn_accelfield in fn_accelfields:
    tmp = load_from_h5(fn_accelfield)
    date = list(tmp['constancy_accel'].keys())[0] #single entry
    accelfields[date] = tmp['constancy_accel'][date]

In [ ]:
ss.keys()

In [ ]:
ss_test0 = ss['test0']
pm =  ss['test0_params']
config = ss['globalconfig']
params = ss['globalparams']
sr = config['audio']['rate']
print(pm)
print(params)
print(config)

In [ ]:
p2obs = lambda p: pfield2obs(p, startx=pm['startx'], stopx=pm['stopx'], nx=pm['nx'], starty=pm['starty'], stopy=pm['stopy'], ny=pm['ny'], rho=params['rho'])

In [ ]:
del ss

In [ ]:
keys_raw =  sorted([k for k in ss_test0.keys() if "Conditioned" not in k]) # nsamples, ny, nx, repeats
keys_with_cond = sorted([k for k in ss_test0.keys() if "Conditioned" in k]) # nsamples, ny, nx, repeats, ny(target), nx(target)

In [ ]:
keys_raw

In [ ]:
keys_with_cond

In [ ]:
plim = 60 # pressure limit
alim = 1.8

In [ ]:
# Constancy over time
for k in keys_with_cond[:1]:
    title = k+f"_CLOSED_LOOP_pressure"
    data = ss_test0[k].mean(3)
    print(title)
    titlebr = linebreaks(title, group=50)
    nsp = data.shape[0]
    ny = data.shape[1]
    nx = data.shape[2]


    ####WAVEFORMS
    fig = plt.figure(figsize=(48 / 2.54, 48 * nx / 2.54 / ny))
    gs = gridspec.GridSpec(ny, nx)
    # plot
    for i in range(ny):
        for j in range(nx):
            ax = fig.add_subplot(gs[i, j])
            ax.plot(1000. * np.arange(nsp, dtype=float) / sr, data[:,i,j,i,j], color='k', lw=.5, label="Initial")
            ax.spines['right'].set_visible(False)
            ax.spines['top'].set_visible(False)
            ax.set_xlabel('Time [ms]')
            ax.set_ylim([-plim, plim])
            ax.set_ylabel("[Pa]")
            tmptitle = f"x={j}, y={i}"
            ax.set_title(tmptitle)
            for date in pfields.keys():
                ax.plot(1000. * np.arange(nsp, dtype=float) / sr, pfields[date][k].mean(3)[:,i,j], lw=.5, alpha=.5, label=date)
            ax.legend(fontsize=8)

    #fig.subplots_adjust(top=0.97)
    fig.suptitle(titlebr, fontsize=8)
    fig.tight_layout(rect=[0, 0, 1, 0.97])
    fn = os.path.join(out,title+'.png')
    fig.savefig(fn, dpi=300)

    ####SCORES
    for date in pfields.keys():
        corrlim = [.8,1]
        nrmselim = [0,10]
        title =  k+f"_CLOSED_LOOP_SCORES_{plot_obs}_{date}"
        titlebr = linebreaks(title, group=50)
        correlations = np.zeros(data.shape[1:3])
        nrmse = correlations.copy()
        for i in range(correlations.shape[0]):
            for j in range(correlations.shape[1]):
                orig =  data[:,i,j,i,j]
                later = pfields[date][k].mean(3)[:,i,j]
                correlations[i,j] = pearsonr(later, orig)[0]
                nrmse[i,j] = np.sqrt(np.mean((later-orig)**2))/(max(orig)-min(orig))
        fig, ax = plt.subplots(1,2)#figsize=(12,8)
        tmp = ax[0].imshow(correlations, vmin=corrlim[0], vmax=corrlim[1], cmap= 'gray')
        colorbar(tmp, "Pearson Correlation")
        tmp = ax[1].imshow(100*nrmse, vmin=nrmselim[0], vmax=nrmselim[1], cmap='gray_r')
        colorbar(tmp, "NRMSE (%)")

        # fig.subplots_adjust(top=0.97)
        fig.suptitle(titlebr, fontsize=8)
        fig.tight_layout(rect=[0, 0, 1, 1.05])
        fn = os.path.join(out,title+'.png')
        fig.savefig(fn, dpi=300)

In [ ]:
# Compare initial pressure gradient to all later PCB acceleration measurements
label = "PCBvsHydrophone"
plot_obs = "acceleration_z"
scale_p = 0.4#0.5

for k in keys_with_cond[:1]:
    title = k+f"_CLOSED_LOOP_{label}_{plot_obs}"
    print(title)
    titlebr = linebreaks(title, group=50)
    nsp = data.shape[0]
    ny = data.shape[1]
    nx = data.shape[2]
    data = ss_test0[k].mean(3)
    
    ####WAVEFORMS
    fig = plt.figure(figsize=(48 / 2.54, 48 * nx / 2.54 / ny))
    gs = gridspec.GridSpec(ny, nx)
    # plot
    for i in range(ny):
        for j in range(nx):
            ax = fig.add_subplot(gs[i, j])
            ax.spines['right'].set_visible(False)
            ax.spines['top'].set_visible(False)
            ax.set_xlabel('Time [ms]')
            ax.set_ylim([-alim, alim])
            ax.set_ylabel("[m/s^2]")
            if plot_obs != "acceleration_z":
                ax.plot(1000. * np.arange(nsp, dtype=float) / sr, scale_p*p2obs(data[:,:,:,i,j])[plot_obs][:,i,j], color='k', lw=.5, label=f"Initial - dp, x {scale_p}")
            tmptitle = f"x={j}, y={i}"
            ax.set_title(tmptitle)
            
            for date in accelfields.keys():
                accelfield = accelfields[date][k].mean(3)
                obs = accelfield2obs(accelfield)
                ax.plot(1000. * np.arange(nsp, dtype=float) / sr, obs[plot_obs][:,i,j], lw=.5, alpha=.5, label=date + " - PCB")
            ax.legend(fontsize=8)

    #fig.subplots_adjust(top=0.97)
    fig.suptitle(titlebr, fontsize=8)
    fig.tight_layout(rect=[0, 0, 1, 0.97])
    fn = os.path.join(out,title+'.png')
    fig.savefig(fn, dpi=300)
    
    
    ####SCORES
    if plot_obs != "acceleration_z":
        for date in accelfields.keys():
            corrlim = [.8,1]
            nrmselim = [0,10]
            title =  k+f"_CLOSED_LOOP_SCORES_{plot_obs}_{date}"
            titlebr = linebreaks(title, group=50)
            correlations = np.zeros(data.shape[1:3])
            nrmse = correlations.copy()
            for i in range(correlations.shape[0]):
                for j in range(correlations.shape[1]):
                    orig = scale_p* p2obs(data[:,:,:,i,j])[plot_obs][:,i,j]
                    later = accelfield2obs(accelfields[date][k].mean(3))[plot_obs][:,i,j]
                    correlations[i,j] = pearsonr(later, orig)[0]
                    nrmse[i,j] = np.sqrt(np.mean((later-orig)**2))/(max(orig)-min(orig))
            fig, ax = plt.subplots(1,2)#figsize=(12,8)
            tmp = ax[0].imshow(correlations, vmin=corrlim[0], vmax=corrlim[1], cmap= 'gray')
            colorbar(tmp, "Pearson Correlation")
            tmp = ax[1].imshow(100*nrmse, vmin=nrmselim[0], vmax=nrmselim[1], cmap='gray_r')
            colorbar(tmp, "NRMSE (%)")

            # fig.subplots_adjust(top=0.97)
            fig.suptitle(titlebr, fontsize=8)
            fig.tight_layout(rect=[0, 0, 1, 1.05])
            fn = os.path.join(out,title+'.png')
            fig.savefig(fn, dpi=300)